In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader

In [3]:
eco_mapping_df = pd.read_csv("../data/ECO_codes_mapping.csv")
eco_mapping = {eco: idx for idx, eco in enumerate(eco_mapping_df["eco"].unique())}

In [4]:
len(eco_mapping.keys())

496

In [5]:
# Load dataset
df = pd.read_csv(r"../data/preprocessed/lichess_games.csv")
# df["matrices"] = df["matrices"].apply(eval)  # Convert string to list

In [11]:
df.describe

<bound method NDFrame.describe of                                                      pgn  \
1      [Event "?"]\n[Site "Lichess.org"]\n[Date "?"]\...   
2      [Event "?"]\n[Site "Lichess.org"]\n[Date "?"]\...   
3      [Event "?"]\n[Site "Lichess.org"]\n[Date "?"]\...   
4      [Event "?"]\n[Site "Lichess.org"]\n[Date "?"]\...   
6      [Event "?"]\n[Site "Lichess.org"]\n[Date "?"]\...   
...                                                  ...   
20051  [Event "?"]\n[Site "Lichess.org"]\n[Date "?"]\...   
20052  [Event "?"]\n[Site "Lichess.org"]\n[Date "?"]\...   
20055  [Event "?"]\n[Site "Lichess.org"]\n[Date "?"]\...   
20056  [Event "?"]\n[Site "Lichess.org"]\n[Date "?"]\...   
20057  [Event "?"]\n[Site "Lichess.org"]\n[Date "?"]\...   

                                                matrices opening_eco  \
1      [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         B00   
2      [[[-4, -2, -3, -5, -6, -3, -2, -4], [-1, -1, -...         C20   
3      [[[-4, -2, -3, -5, -6,

In [12]:
df["opening_eco"].value_counts()

opening_eco
A00    1007
C00     844
D00     739
B01     716
C41     691
C20     675
A40     618
B00     611
B20     567
C50     538
C40     446
D02     434
C44     375
C42     327
C55     312
B07     304
A04     285
C45     284
C02     277
C46     267
A45     252
B21     237
B30     227
B50     226
D20     211
Name: count, dtype: int64

In [10]:
RARE_OPENINGS = 200
eco_counts = df["opening_eco"].value_counts()
rare_classes = eco_counts[eco_counts < RARE_OPENINGS].index
df = df[~df["opening_eco"].isin(rare_classes)]

In [4]:
df["matrices"][0]

[[[-4, -2, -3, -5, -6, -3, -2, -4],
  [-1, -1, -1, -1, -1, -1, -1, -1],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 0, 1, 1, 1, 1],
  [4, 2, 3, 5, 6, 3, 2, 4]],
 [[-4, -2, -3, -5, -6, -3, -2, -4],
  [-1, -1, -1, 0, -1, -1, -1, -1],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, -1, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 0, 1, 1, 1, 1],
  [4, 2, 3, 5, 6, 3, 2, 4]],
 [[-4, -2, -3, -5, -6, -3, -2, -4],
  [-1, -1, -1, 0, -1, -1, -1, -1],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, -1, 0, 0, 0, 0],
  [0, 0, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 0, 0, 1, 1, 1, 1],
  [4, 2, 3, 5, 6, 3, 2, 4]],
 [[-4, -2, -3, -5, -6, -3, -2, -4],
  [-1, -1, 0, 0, -1, -1, -1, -1],
  [0, 0, -1, 0, 0, 0, 0, 0],
  [0, 0, 0, -1, 0, 0, 0, 0],
  [0, 0, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 0, 0, 1, 1, 1, 1],
  [4, 2, 3, 5, 6, 3, 2, 4]],
 [[-4, -2, -3, -5, -6, -3, -2, -4],


In [6]:
# Encode ECO labels
eco_labels = sorted(df["opening_eco"].unique())
eco_to_idx = {eco: idx for idx, eco in enumerate(eco_labels)}
idx_to_eco = {idx: eco for eco, idx in eco_to_idx.items()}

In [ ]:
# Dataset class
class ChessOpeningDataset(Dataset):
    def __init__(self, df):
        self.data = df
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        matrices = row["matrices"]
        opening_ply = row["opening_ply"]
        eco = row["opening_eco"]
        
        # Remove the first 2 * opening_ply + random number of moves
        num_remove = 2 * opening_ply + random.randint(0, 5)
        matrices = matrices[num_remove:] if len(matrices) > num_remove else matrices
        
        # Convert to tensor (pad if necessary)
        matrices = [torch.tensor(m, dtype=torch.float32) for m in matrices]
        if len(matrices) == 0:
            matrices = [torch.zeros((8, 8), dtype=torch.float32)]  # Ensure at least one matrix
        
        matrices = torch.stack(matrices)  # Shape: (T, 8, 8)
        label = torch.tensor(eco_to_idx[eco], dtype=torch.long)
        
        return matrices, label

In [ ]:
# Define CNN model
class ChessOpeningClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ChessOpeningClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        x = self.relu(self.conv1(x))  # (B, 32, 8, 8)
        x = self.pool(self.relu(self.conv2(x)))  # (B, 64, 4, 4)
        x = x.view(x.shape[0], -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = ChessOpeningDataset(df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ChessOpeningClassifier(len(eco_labels)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for matrices, labels in dataloader:
        matrices, labels = matrices.to(device), labels.to(device)
        matrices = matrices.unsqueeze(1)  # Add channel dim (B, 1, 8, 8)
        
        optimizer.zero_grad()
        outputs = model(matrices)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")

In [ ]:
# Save model
torch.save(model.state_dict(), r"/models/chess_opening_model.pth")